<a href="https://colab.research.google.com/github/Sachin-RECB/Hospital_Nav/blob/main/HospitalNavPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import folium
from folium import plugins
import pandas as pd

import ipywidgets
import os
import json

from IPython.display import YouTubeVideo
import datetime

ModuleNotFoundError: No module named 'folium'

## To learn the basics about geo visualisation in Python check out this Video about the cholera outbreak in broad street

In [ ]:
!git clone https://github.com/Sachin-RECB/Hospital_nav.git


---
## Intitialize Map
---

In [ ]:
# the latitude and Longitude coordinates
UMMlocation = (29.13246860514598, 78.24661892659645)
map_UMM = folium.Map(location = UMMlocation, width = "75%", zoom_start = 17) # max zoom: 18
map_UMM

---
## Display geojson Outline
---
geojson drawing tool: http://geojson.io/

In [ ]:
hauseOutline = 'HospitalNavigator/GeoResources/BH2.geojson'
display(folium.GeoJson(hauseOutline, name="BH2").add_to(map_UMM))
display(map_UMM)

In [ ]:
hauseOutline = 'HospitalNavigator/GeoResources/Haus22.geojson'
display(folium.GeoJson(hauseOutline, name="Haus22").add_to(map_UMM))
display(map_UMM)

---
##Display Ant Path
---

In [ ]:
testGeoJson = 'Hospital_nav/GeoResources/path/w17.geojson'

In [ ]:
def switchPosition(coordinate):
  temp = coordinate[0]
  coordinate[0] = coordinate[1]
  coordinate[1] = temp
  return coordinate

In [ ]:
with open(testGeoJson) as f:
  testWay = json.load(f)

for feature in testWay['features']:
    path = feature['geometry']['coordinates']
finalPath = list(map(switchPosition,path))
finalPath

In [ ]:
path = 'Hospital_nav/GeoResources/path/w17.geojson'
folium.plugins.AntPath([[29.132290662160102, 78.24655887636258],
 [29.132952121789103, 78.24860610961012],
 [29.133244393292657, 78.24845641943719],
 [29.132879053783363, 78.2463255357989],
 [29.132267587910167, 78.24588967323652],
 [29.131729187274967, 78.24692429649065],
 [29.132759837451996, 78.24804257013552],
 [29.1322945078679, 78.24774318978965]]).add_to(map_UMM)
map_UMM

---
## Select Widget
---

In [ ]:
select_widget=ipywidgets.Select(
    options=['Option A', 'Option B'],
    value='Option A',
    description='Select',
    disabled=False)

def selectOption(opt):
    if opt == 'Option A':
        print('A')
    if opt == 'Option B':
        print('B')

ipywidgets.interact(selectOption, opt=select_widget)



---
## Build Navigator Prototype
---




In [ ]:
class navigator:
    def __init__(self):
        self.geoResources = {}
        self.hospitalLocation =(29.13246860514598, 78.24661892659645)
        self.position = 'w'
        self.destination = 'Haus17'

        for root, dirs, files in os.walk('HospitalNavigator/GeoResources'):  
            for file in files:
                self.geoResources[file.split('.')[0]] = root+'/'+file

    def changeDestination(self,newDestination):
        self.destination = newDestination
        self.redrawMap()

    def changeStartPoint(self, newStartPoint):
        
        #self.position = newStartPoint #does not work yet
        print(f'Selected Start: {newStartPoint}; Selected Target: {self.destination}')
        #self.redrawMap()
        

    def drawPathWay(self,hospitalMap):
      
      def switchPosition(coordinate):
        temp = coordinate[0]
        coordinate[0] = coordinate[1]
        coordinate[1] = temp
        return coordinate

      searchString = self.position + self.destination.split('Haus')[1]
      with open(self.geoResources[searchString]) as f:
           testWay = json.load(f)

      for feature in testWay['features']:
        path = feature['geometry']['coordinates']

      finalPath = list(map(switchPosition,path))
      folium.plugins.AntPath(finalPath).add_to(hospitalMap)

    def drawBuilding(self,hospitalMap):
      hauseOutline = self.geoResources[self.destination]
      folium.GeoJson(hauseOutline, name="geojson").add_to(hospitalMap)

    def redrawMap(self):
        #print(f'position {self.position}, destination {self.destination}')
        hospitalMap = folium.Map(location = self.hospitalLocation, width = "75%", zoom_start = 17)
        self.drawPathWay(hospitalMap)
        self.drawBuilding(hospitalMap)
        display(hospitalMap)




In [ ]:
myNavigator = navigator()

In [ ]:
def displayWay(whereTo):
     myNavigator.changeDestination(whereTo)

In [ ]:
def changePosition(whereFrom):
    myNavigator.changeStartPoint(whereFrom)

In [ ]:
# Destination Selector
selectHouse_widget=ipywidgets.Select(
    
options=['Main building',
    'BH1',
    'BH2',
    'BH4',
    'Girls Hostel',
    'CBI ATM',
    'Volleyball Court',
    'Activity Hall'],
    value='Main Building',
    description='Target',
    disabled=False)

# widget function
def selectHouse(way):
    if way == 'Main Building' :
        displayWay('Main Building' ) 
    if way == 'BH1':
        displayWay('BH1')
    if way == 'BH2':
        displayWay('BH2')
    if way == 'BH4':
        displayWay('BH4')
    if way == 'Girls Hostel':
        displayWay('Girls Hostel')
    if way == 'CBI ATM':
        displayWay('CBI ATM')
    if way == 'Volleyball Court':
        displayWay('Volleyball Court')
    if way == 'Activity Hall':
        displayWay('Activity Hall') 

# Position Selector
selectPosition_widget=ipywidgets.Select(
    options=['Aufnahme', 'Eingang West', 'Eingang Ost', 'Eingang Fahrzeuge'],
    value='Eingang West',
    description='Start',
    disabled=False)

def selectPosition(position):
    if position == 'Aufnahme':
        changePosition('a')
    if position == 'Eingang West':
        changePosition('w')
    if position == 'Eingang Ost':
        changePosition('o')
    if position == 'Eingang Fahrzeuge':
        changePosition('f')
        
#Initialization   
ipywidgets.interact(selectPosition, position=selectPosition_widget)
ipywidgets.interact(selectHouse, way=selectHouse_widget)